In [4]:
### Prerequisties
# most of the packages are available via anaconda
# some of them require to be installed separately

# some of the packages (especially nltk) are necessary only for preprocessing
# feel free to ignore them and move directly into later sections

import os
import pandas as pd
import numpy as np
import nltk
import pickle
import json
import sys
import re

import csv
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

from gensim import corpora, models, similarities
### drawing on a basic tutorial: https://radimrehurek.com/gensim/tut1.html#corpus-streaming-one-document-at-a-time
### we use it here basically only for generating the vector model

import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer

### for plotting
import matplotlib.pyplot as plt
### import cufflinks as cf - to connect plotly and pandas

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe


from google.oauth2 import service_account # based on google-auth library
import sddk
# a few other packages are imported in the section on visualizing distances

In [5]:
conf = sddk.configure()

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
endpoint variable has been configured to: https://sciencedata.dk/files/


In [6]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get(conf[1] + "ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url

In [7]:
CSEL_authors_JK = get_as_dataframe(gc.open_by_url("https://docs.google.com/spreadsheets/d/1F9EKced2JVW4a34Wu3XYLZ-kaZGP6Qt3Nh0GQjdp3to/edit?usp=sharing").worksheet("Authors"))
CSEL_authors_JK.head(5)

,ID,Author,ID_stoa,Honorific,Date of birth,Date of death,Active period middle,Activity date approximate,Active since,Active until,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,1.0,Adamnan,stoa0007,saint,625,704,"=ROUND((E2+20+F2)/2, 2)",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10.0,Cyprian of Carthage,stoa0104a,saint,200,258,=(E3+20+F3)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11.0,Cyprianus Gallus,stoa0104c,NaN,397,430,=(E4+20+F4)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.0,Ennodius Magnus Felix,stoa0114a,saint,474,521,=(E5+20+F5)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13.0,Eucherius of Lyon,stoa0117,saint,380,449,=(E6+20+F6)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
### upload my own (VK) biblical abbreviatians as a tuple

abbr_kase_df = get_as_dataframe(gc.open_by_url('https://docs.google.com/spreadsheets/d/1vWUsY1moh6b5A27YDxPSuibayXNLRUV97Hz9TICyBBo/edit?usp=sharing').worksheet("replace_abbr"))
abbr_kase_df.head(5)

,0,1
0,Gn,Gen.
1,Genes.,Gen.
2,Ex.,Exo.
3,Lv.,Lev.
4,Nm.,Num.


In [ ]:
repl_tuple_VK = list(abbr_kase_df.itertuples(index=False, name=None))
print(repl_tuple_VK)

In [0]:
### to work with GoogleSheets

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [11]:
### path in the VK drive - other users have to modify accordingly...

CSEL_raw_df = sddk.read_file("AUCT/data/CSEL_raw.csv", "df", conf)
#names=["file_name", "author_from_xml_tag", "title", "volume","text"])
CSEL_raw_df.head(5)

In [0]:
### authors from JarKub
CSEL_authors_JK = get_as_dataframe(gc.open_by_url("https://docs.google.com/spreadsheets/d/1F9EKced2JVW4a34Wu3XYLZ-kaZGP6Qt3Nh0GQjdp3to/edit?usp=sharing").worksheet("Authors"))
CSEL_authors_JK.head(5)

,ID,Author,ID_stoa,Honorific,Date of birth,Date of death,Active period middle,Activity date approximate,Active since,Active until,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,1.0,Adamnan,stoa0007,saint,625,704,=(E2+20+F2)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10.0,Cyprian of Carthage,stoa0104a,saint,200,258,=(E3+20+F3)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11.0,Cyprianus Gallus,stoa0104c,NaN,397,430,=(E4+20+F4)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12.0,Ennodius Magnus Felix,stoa0114a,saint,474,521,=(E5+20+F5)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13.0,Eucherius of Lyon,stoa0117,saint,380,449,=(E6+20+F6)/2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
### adding Kuben's author names to our data

def author_from_file_name(file_name):
  try:
    author = CSEL_authors_JK.loc[CSEL_authors_JK["ID_stoa"]==file_name.partition(".")[0], "Author"].values[0]
  except:
    author = "unknown" 
  return author
CSEL_raw_df["author"] = CSEL_raw_df.apply(lambda row: author_from_file_name(row["file_name"]) , axis=1)


In [0]:
# one file for testing purposes, let's extract one file

one_text_df = CSEL_raw_df[CSEL_raw_df["file_name"]=="stoa0076c.stoa001.opp-lat2.xml"].copy()
one_text_df

,file_name,author_from_xml_tag,title,volume,text,author
243,stoa0076c.stoa001.opp-lat2.xml,Iohannes Cassianus,Conlationes,unknown,"\n\n\n\n\nPRAEFATIO. \n\n\nDebitum, quod beati...",Cassian John


In [0]:
### upload my own (VK) biblical abbreviatians as a tuple

abbr_kase_df = get_as_dataframe(gc.open_by_url('https://docs.google.com/spreadsheets/d/1vWUsY1moh6b5A27YDxPSuibayXNLRUV97Hz9TICyBBo/edit?usp=sharing').worksheet("replace_abbr"))
repl_tuple_VK = list(abbr_kase_df.itertuples(index=False, name=None))
print(repl_tuple_VK)

[('Gn ', 'Gen. '), ('Genes.', 'Gen.'), ('Ex.', 'Exo.'), ('Lv.', 'Lev.'), ('Nm.', 'Num.'), ('Dt.', 'Deut'), ('I Sm.', '1Sam.'), ('1Sm.', '1Sam.'), ('1 Sam.', '1Sam.'), ('II Sm.', '2Sam.'), ('2 Sam.', '2Sam.'), ('Ios.', 'Josh.'), ('Idc.', 'Judg.'), ('Jdg.', 'Judg.'), ('1 Reg.', '1Kgs.'), ('II Reg.', '2Kgs.'), ('2 Reg.', '2Kgs.'), ('2 Rg.', '2Kgs.'), ('II Rg.', '2Kgs.'), ('2Rg.', '2Kgs.'), ('2 Ki.', '2Kgs.'), (' II Ki.', '2Kgs.'), ('I Reg.', '1Kgs.'), ('1 Rg.', '1Kgs.'), ('I Rg.', '1Kgs.'), ('1Rg.', '1Kgs.'), ('1 Ki.', '1Kgs.'), ('I Ki.', '1Kgs.'), ('2 Ch.', '2Chr.'), ('II Ch.', '2Chr.'), ('2 Chr.', '2Chr.'), ('II Chr.', '2Chr.'), ('2 Par.', '2Chr.'), ('II Par.', '2Chr.'), ('1 Ch.', '1Chr.'), ('I Ch.', '1Chr.'), ('1 Chr.', '1Chr.'), ('I Chr.', '1Chr.'), ('1 Par.', '1Chr.'), ('I Par.', '1Chr.'), ('Est.', 'Esth.'), ('Hiob', 'Job'), ('Iob', 'Job'), ('PB.', 'Psa.'), ('Psal.', 'Psa.'), ('Ps.', 'Psa.'), ('Ecc.', 'Eccl.'), ('Ecl.', 'Eccl.'), ('Prov.', 'Pro.'), ('Prou.', 'Pro.'), ('Prv.', 'Pro.')

In [0]:
### check their length
len(repl_tuple_VK)

151

In [0]:
### upload abbreviatians variants as defined by Mr. Kuben (into a dataframe)
abbr_kuben = get_as_dataframe(gc.open_by_url("https://docs.google.com/spreadsheets/d/15_acFNFaTUgTW4DODk4hdMcgvGeyQlTbx783gTrEXhI/edit?usp=sharing").worksheet("List 1"))
### upload the biblical abbr converter (from DZ)
abbr_conv_df = get_as_dataframe(gc.open_by_key("1byVbRwnQ058--0ud1_NBt2xgIhuiV7LK1BmCm67CTxA").worksheet("AbbrevConverter"))


### use the converter to transform Mr. Kuben's abbrevatians into the tuple format with "CSEL_VK"
repl_tuple_JK = []
for row in abbr_kuben.values.tolist(): 
  if row[0]>0: ### i.e. if it has valid (i.e. numerical) index value
    replaced_abbr = abbr_conv_df.loc[abbr_conv_df["Book_ID"]==float(row[0]), "CSEL_VK"].values[0]
    for element in row[2:]: ### for each cell on the row
      if "nan" not in str(element): ### i.e. if the cell content is not "nan" 
        tuple_pair = (element, replaced_abbr)
        repl_tuple_JK.append(tuple_pair)
print(repl_tuple_JK)



[('Gen.', 'Gen.'), ('Genes.', 'Gen.'), ('Gon.', 'Gen.'), ('Qen.', 'Gen.'), ('Oen.', 'Gen.'), ('Ex.', 'Exo.'), ('Exod.', 'Exo.'), ('Ei.', 'Exo.'), ('E*.', 'Exo.'), ('Er.', 'Exo.'), ('EL', 'Exo.'), ('Eiod.', 'Exo.'), ('Ezod.', 'Exo.'), ('Levit.', 'Lev.'), ('Leuit.', 'Lev.'), ('Lev.', 'Lev.'), ('LeY.', 'Lev.'), ('Leu. ', 'Lev.'), ('Len.', 'Lev.'), ('Leoit.', 'Lev.'), ('Num.', 'Num.'), ('Nam.', 'Num.'), ('Nom.', 'Num.'), ('NUID.', 'Num.'), ('Nurn.', 'Num.'), ('NumBr.', 'Num.'), ('Nomer.', 'Num.'), ('Nuin.', 'Num.'), ('Deut.', 'Deut.'), ('Dent.', 'Deut.'), ('Deot.', 'Deut.'), ('Deat.', 'Deut.'), ('Deuteron.', 'Deut.'), ('Deuter.', 'Deut.'), ('Deuteroii.', 'Deut.'), ('Deater.', 'Deut.'), ('Uenter.', 'Deut.'), ('neuter.', 'Deut.'), ('Jos.', 'Josh.'), ('Ios.', 'Josh.'), ('los.', 'Josh.'), ('Jud.', 'Judg.'), ('Iud.', 'Judg.'), ('Ind.', 'Judg.'), ('Iad.', 'Judg.'), ('I Sam. ', '1Sam.'), ('II Sam.', '2Sam.'), ('I Reg.', '1Kgs.'), ('III Reg.', '1Kgs.'), ('m Beg.', '1Kgs.'), ('III Beg.', '1Kgs.'), 

In [0]:
### number of abbr as defined by Mr. Kuben
len(repl_tuple_JK)

348

In [0]:
### combinqtion of VK's and JK's abbrs

repl_tuple_all = repl_tuple_VK.copy()
for tuple_pair in repl_tuple_JK:
  if tuple_pair not in repl_tuple_all:
    repl_tuple_all.append(tuple_pair)
len(repl_tuple_all)

454

In [0]:
### FUNCTION: replace Roman numbers with arabic

def roman_to_int(n):
    n = str(n.upper())
    numeral_map = zip(
        (1000, 900, 500, 400, 100, 90, 50, 40, 10, 9, 5, 4, 1),
        ('M', 'CM', 'D', 'CD', 'C', 'XC', 'L', 'XL', 'X', 'IX', 'V', 'IV', 'I')
    )
    i = 0
    result = 0
    for integer, numeral in numeral_map:
        while n[i:i + len(numeral)] == numeral:
            result += integer
            i += len(numeral)  
    return result
  
# go throug every note one by one
def find_all_abbr_data(dataframe, repl_tuple, expand):
  expand_ranges = expand ### set to False if you dont like expanded data
  bib_abbr = []
  abbrs_ = [element.replace(" ", "_") for element in [tup[0] for tup in repl_tuple]]
  abbrs_final = [tup[1] for tup in repl_tuple]
  for file_name, author, title, text in zip(dataframe["file_name"], dataframe["author"], dataframe["title"], dataframe["text"]):
      # remove residua of our older regex application
      for element, abbr_ in zip([tup[0] for tup in repl_tuple], abbrs_):
        text = text.replace(element, abbr_)
      author_text_split = text.split()
      # find all matches between words in the text and abbr. of biblical books
      for num, word in enumerate(author_text_split):
          if word in abbrs_:
              word = word.replace("_", " ") # change it back into the original form
              word_plus_one = author_text_split[num+1]
              word_plus_two = author_text_split[num+2]
              word_minus_one = author_text_split[num-1]
              try: ### sometimes the third word is not available, probably because of the end of the document
                word_plus_three = author_text_split[num+3]
                words_original = " ".join([word, word_plus_one, word_plus_two, word_plus_three])
              except:
                words_original = " ".join([word, word_plus_one, word_plus_two])
              for element in repl_tuple:
                if word == element[0]:
                  word = element[1]
                  break
              export_for_TH = " ".join([word, word_plus_one, word_plus_two])
              if (word_plus_one == "cap.") or (word_plus_one == "c."):
                  word_plus_one = author_text_split[num+2]
                  word_plus_two = author_text_split[num+3]
              try: ### identify any roman number in chapters and replace them by arabic
                  found = re.search(r'(^(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})(\b))', word_plus_one, flags=re.IGNORECASE).groups()[0]
                  if not found == "":
                      word_plus_one = word_plus_one.replace(found, str(roman_to_int(found)))
              except:
                  pass
              try: 
                  chapterverse = re.search('(\d+)(\.|\,|\:)(\d+)', word_plus_one, flags=re.IGNORECASE).groups()
                  word_plus_one = chapterverse[0]
                  word_plus_two = chapterverse[2]
              except:
                  pass
              if not word_plus_one[0].isdigit():
                  word_plus_one = ""
                  precision_type = "no_chapter_"
              else:
                  word_plus_one = re.search(r'(\d+)', word_plus_one, flags=re.IGNORECASE).groups()[0]
                  precision_type = "chapter_"
              if word_plus_two.startswith("v."):
                  word_plus_two.replace("v.", "")
              if not word_plus_two[0].isdigit():
                  word_plus_two = ""
                  precision_type = precision_type + "no_verse"
              else:
                  word_plus_two = re.search(r'((\d+)(\-\d+)?)', word_plus_two, flags=re.IGNORECASE).groups()[0]
                  precision_type = precision_type + "verse"
              try: ### there is some problem, making this not applicable to all rows
                genre = abbr_conv_df.loc[abbr_conv_df["CSEL_VK"]==word, "Classification_genre_DZ1"].values[0]
                testament = abbr_conv_df.loc[abbr_conv_df["CSEL_VK"]==word, "Classification_testament"].values[0]
                czech = abbr_conv_df.loc[abbr_conv_df["CSEL_VK"]==word, "Czech"].values[0]
              except:
                pass
              if word_minus_one.lower() == "cf.":
                  citation_or_allusion = "a"
              else:
                  citation_or_allusion = "c"
              more_verses = re.match(r'(\d+)(\-)(\d+)', word_plus_two)
              ### to match instances with more verses (Gen. 5.7-14)
              if more_verses is not None:
                beginning = int(more_verses.groups()[0])    
                end = int(more_verses.groups()[2])
                words_modified = "_".join([word.replace(".","").lower(), word_plus_one, str(beginning) + "to" + str(end)])  
                if expand_ranges == True:
                    beginning = int(more_verses.groups()[0])
                    end = int(more_verses.groups()[2])
                    words_modified = "_".join([word.replace(".","").lower(), word_plus_one, str(beginning) + "to" + str(end)])
                    actual_verse = beginning
                    while (actual_verse <= end):
                      bib_abbr.append([file_name, author, title, word_minus_one, words_original, export_for_TH, citation_or_allusion, words_modified, "ranged", precision_type, word, word_plus_one, actual_verse, beginning, end, czech, genre, testament])                 
                      actual_verse = actual_verse + 1
                else:
                    bib_abbr.append([file_name, author, title, word_minus_one, words_original, export_for_TH, citation_or_allusion, words_modified, "ranged", precision_type, word, word_plus_one, str(beginning) + "-" + str(end), beginning, end, czech, genre, testament])                 
              else:
                  words_modified = "_".join([word.replace(".","").lower(), word_plus_one, word_plus_two])
                  bib_abbr.append([file_name, author, title, word_minus_one, words_original, export_for_TH, citation_or_allusion, words_modified, "single", precision_type, word, word_plus_one, word_plus_two, None, None, czech, genre, testament])                 
  bib_abbr_df = pd.DataFrame(bib_abbr, columns=["file_name", "author", "Work", "word_before", "words_original", "text_as_found", "Citation_or_allusion", "condenced_expression", "Composite_type", "completion?", "CSEL_VK", "chapter", "verse", "range_firstverse", "range_lastverse", "Czech", "Classification_genre_DZ1", "Classification_testament"])   
  return bib_abbr_df

In [0]:
###start communicate with the google sheet
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1vWUsY1moh6b5A27YDxPSuibayXNLRUV97Hz9TICyBBo/edit?usp=sharing')


In [0]:
### to produce data in a condense form, i.e. without expanded ranges

CSEL_abbr_NEW = find_all_abbr_data(CSEL_raw_df, repl_tuple_JK, False)
len(CSEL_abbr_NEW)

58542

In [0]:
sh.add_worksheet(title="CSEL_abbr_APR23", rows=1, cols=10)
set_with_dataframe(sh.worksheet("CSEL_abbr_APR23"), CSEL_abbr_NEW)

In [0]:
### to produce data in an expanded form, i.e. with ranges

CSEL_abbr_NEW_expanded = find_all_abbr_data(CSEL_raw_df, repl_tuple_JK, True)
len(CSEL_abbr_NEW_expanded)

72586

In [0]:
sh.add_worksheet(title="CSEL_abbr_APR23_expanded", rows=1, cols=10)
set_with_dataframe(sh.worksheet("CSEL_abbr_APR23_expanded"), CSEL_abbr_NEW_expanded)